# Colour Correction using Splines

### Define constants

In [1]:
%load_ext autoreload
%autoreload 2
import colour
from data import load_dataset_sfu, load_dataset_csv, load_cmfs, load_camera, load_insitu, msds_to_rgb, msds_to_xyz, load_dataset_skin
import numpy as np

RANDOM_STATE = 0
np.random.seed(RANDOM_STATE)
    
SFU_FILE_PATH = 'data/reflect_db.reflect'
CAVE_FOSTER2004_PATH = 'data/cave_foster2002.csv'
INSITU_PATH = "data/insitu_dataset.csv"
CAVE_PATH = 'data/cave.csv'
FOSTER_50_PATH = 'data/foster50.csv'
CAMERA = 'sigma'

In [2]:

# sfu_dataset = load_dataset_sfu(SFU_FILE_PATH)
cave_foster2004_dataset = load_dataset_csv(CAVE_FOSTER2004_PATH)
foster_50_dataset = load_dataset_csv(FOSTER_50_PATH)
insitu = load_insitu(INSITU_PATH)
# cave = load_dataset_csv(CAVE_PATH)
# skin = load_dataset_skin()


In [3]:
from plotting import plot_chromaticity_diagram
TRAIN = insitu
TEST = foster_50_dataset
VALIDATION = insitu

### Computing Observer Responses
We can easily change the order of test and train sets here

In [4]:
from plotting import plot_chromaticity_diagram

cmfs, illuminant = load_cmfs()
response_trainset_xyz = msds_to_xyz(TRAIN, cmfs, illuminant)
response_testset_xyz = msds_to_xyz(TEST, cmfs, illuminant)
response_validation_xyz = msds_to_xyz(VALIDATION, cmfs, illuminant)

[ 0.94940092  1.          1.08709122]
[ 0.94940092  1.          1.08709122]
[ 0.94940092  1.          1.08709122]


### Computing Camera Responses

In [5]:
from utilities import add_noise
MSDS_TRAIN = load_camera(CAMERA)
response_trainset_camera = msds_to_rgb(TRAIN,MSDS_TRAIN, illuminant)
response_testset_camera = msds_to_rgb(TEST,MSDS_TRAIN, illuminant)
response_validationset_camera = msds_to_rgb(VALIDATION,MSDS_TRAIN, illuminant)
noise_gauss = 0.01
noise_poiss = 0.01
response_testset_camera_og = response_testset_camera
response_testset_camera = add_noise(response_testset_camera, noise_poiss, noise_gauss)

[  613.25528366  1055.57702291   900.69359634]
[  613.25528366  1055.57702291   900.69359634]
[  613.25528366  1055.57702291   900.69359634]


### Fit Generalized Additive Model with P-splines

### Nikon

In [17]:
from sklearn.metrics import make_scorer
from colour_math import deltae_mean
from models import GAMOptimizer
from evaluate import pred
np.int = np.int64
if CAMERA == 'nikon':
    gam = GAMOptimizer(lams=1e-6, order=3, n_splines=5)
    
    # gam = GAMOptimizer(lams=0.0001,order=3, n_splines=10)
    # gam = GAMOptimizer(lams=0.01,order=3, n_splines=20)

    gam.fit(response_trainset_camera, response_trainset_xyz)

    pred(gam, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

225
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 9.99
DeltaE max: 58.98
DeltaE median: 7.89
DeltaE 95 percentile: 24.86
DeltaE 99 percentile: 33.74


### Sigma

In [7]:
from sklearn.metrics import make_scorer
from colour_math import deltae_mean
from models import GAMOptimizer
from evaluate import pred
np.int = np.int64
if CAMERA == 'sigma':
    # gam = GAMOptimizer(lams=1e-9, order=3, n_splines=5)
    # gam = GAMOptimizer(lams=0.0001,order=3, n_splines=10)
    gam = GAMOptimizer(lams=0.0001,order=3, n_splines=20)

    gam.fit(response_trainset_camera, response_trainset_xyz)

    pred(gam, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

### Fit Linear Model

In [8]:
from sklearn.linear_model import LinearRegression

linear = LinearRegression(fit_intercept=False)


linear.fit(response_trainset_camera, response_trainset_xyz)
pred(linear, response_testset_camera, response_testset_xyz, "Foster 50")


---- RESULTS Foster 50 ----
DeltaE mean: 10.13
DeltaE max: 58.81
DeltaE median: 8.09
DeltaE 95 percentile: 24.91
DeltaE 99 percentile: 33.73


In [9]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000RP = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=False, degree=1))
])


DE2000RP.fit(response_trainset_camera, response_trainset_xyz)
pred(DE2000RP, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 0.5389760833857261
        x: [ 5.835e-01  3.773e-01 -1.442e-02  2.081e-01  1.102e+00
            -3.130e-01 -2.715e-02 -1.693e-01  1.275e+00]
      nit: 32
      jac: [ 7.461e-05  4.202e-05  2.200e-05 -5.288e-05 -2.566e-05
            -8.665e-06 -4.493e-06 -3.040e-06 -3.226e-06]
 hess_inv: [[ 2.464e-02 -5.539e-02 ..., -5.375e-02  2.959e-02]
            [-5.539e-02  1.714e-01 ...,  1.692e-01 -1.199e-01]
            ..., 
            [-5.375e-02  1.692e-01 ...,  2.639e-01 -1.929e-01]
            [ 2.959e-02 -1.199e-01 ..., -1.929e-01  1.670e-01]]
     nfev: 722
     njev: 71
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 9.93
DeltaE max: 58.83
DeltaE median: 7.86
DeltaE 95 percentile: 24.68
DeltaE 99 percentile: 33.62


### Fit 3rd order Root-Polynomial Model

In [10]:
from models import PolynomialTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

RP_linear_3 = Pipeline([
    ('transformer', PolynomialTransformer(degree=3, rp=True)),
    ('regressor', LinearRegression(fit_intercept=False))
])

RP_linear_3.fit(response_trainset_camera, response_trainset_xyz)

pred(RP_linear_3, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")


---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 12.64
DeltaE max: 122.49
DeltaE median: 8.41
DeltaE 95 percentile: 38.91
DeltaE 99 percentile: 63.69


### Fit a 2nd order Root-Polynomial Model

In [11]:
from models import GAMOptimizer, PolynomialTransformer, DeltaEOptimizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

RP_linear_2 = Pipeline([
    ('transformer', PolynomialTransformer(degree=2, rp=True)),
    ('regressor', LinearRegression(fit_intercept=False))
])


RP_linear_2.fit(response_trainset_camera, response_trainset_xyz)
pred(RP_linear_2, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")


---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 10.04
DeltaE max: 61.44
DeltaE median: 7.93
DeltaE 95 percentile: 24.97
DeltaE 99 percentile: 34.17


In [12]:
P_Linear_3 = Pipeline([
    ('transformer', PolynomialTransformer(degree=3, rp=False)),
    ('regressor', LinearRegression(fit_intercept=False))
])


P_Linear_3.fit(response_trainset_camera, response_trainset_xyz)
pred(P_Linear_3, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 10.08
DeltaE max: 59.04
DeltaE median: 7.99
DeltaE 95 percentile: 25.01
DeltaE 99 percentile: 33.85


In [13]:
P_linear = Pipeline([
    ('transformer', PolynomialTransformer(degree=2, rp=False)),
    ('regressor', LinearRegression(fit_intercept=False))
])


P_linear.fit(response_trainset_camera, response_trainset_xyz)
pred(P_linear, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")


---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 10.08
DeltaE max: 58.82
DeltaE median: 8.03
DeltaE 95 percentile: 24.86
DeltaE 99 percentile: 33.75


In [14]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000P = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=False, degree=3))
])


DE2000P.fit(response_trainset_camera, response_trainset_xyz)
pred(DE2000P, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 0.36060132705
        x: [ 6.045e-01  3.568e-01 ...,  1.271e+00 -1.062e+01]
      nit: 385
      jac: [ 3.422e-05  4.604e-05 ...,  1.144e-06  2.939e-06]
 hess_inv: [[ 1.728e-01 -2.746e-01 ...,  2.888e+00 -1.788e+00]
            [-2.746e-01  6.139e-01 ..., -5.606e+00  1.851e+00]
            ..., 
            [ 2.888e+00 -5.606e+00 ...,  7.373e+02 -9.079e+02]
            [-1.788e+00  1.851e+00 ..., -9.079e+02  1.768e+03]]
     nfev: 27742
     njev: 478
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 9.87
DeltaE max: 58.96
DeltaE median: 7.75
DeltaE 95 percentile: 24.70
DeltaE 99 percentile: 33.70


In [15]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000RP = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=True, degree=3))
])


DE2000RP.fit(response_trainset_camera, response_trainset_xyz)
pred(DE2000RP, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 0.34145898827804777
        x: [-2.184e+00 -1.641e+01 ...,  6.213e+00 -1.173e+01]
      nit: 615
      jac: [ 1.616e-02  8.093e-03 ..., -8.963e-04 -7.421e-04]
 hess_inv: [[ 2.198e+01 -2.163e+01 ..., -1.663e+01  1.351e+01]
            [-2.163e+01  2.562e+02 ..., -3.322e+02  3.443e+02]
            ..., 
            [-1.662e+01 -3.322e+02 ...,  1.108e+03 -6.940e+02]
            [ 1.351e+01  3.443e+02 ..., -6.940e+02  2.836e+03]]
     nfev: 29571
     njev: 739
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 11.72
DeltaE max: 98.92
DeltaE median: 8.10
DeltaE 95 percentile: 34.29
DeltaE 99 percentile: 55.75
